### MACHINE LEARNING PROJECT

To start this project I need a dataset. I couldn't find an apropiate one, but I found [this repository][1] by  
Lucas David that gave me a great starting point.

Using his code I downloaded a big amount of pictures by list of artis with a noticeable style difference:

* Caravaggio
* Edgar Degas
* Francisco de Goya
* Katsushika Hokusai
* Frida Kahlo
* Wassily Kandinsky
* Gustav Klimt
* Roy Lichtenstein
* Piet Mondrian
* Claude Monet
* Pablo Picasso
* Jackson Pollock
* Joaquín Sorolla
* Diego Velazquez
* Andy Warhol

It took me around three hours to get all the XXXXX pictures.

In this notebook I made basic operations with images and define the functions that let me build my own dataset.

[1]: https://github.com/lucasdavid/wikiart

___
### PREREQUISITES

To process images I'll be using OpenCV library. It is important to take a look at the [docs][1] before running the next cell  
as you may want to use another OpenCV package. For this project I'll use the *'main modules package'*

[1]:https://pypi.org/project/opencv-python/

In [101]:
# !pip install opencv-python

In [102]:
### TODO - .yaml

___
### IMPORTS

In [103]:
# Modules used for data handling / test
import os
import csv
import cv2
import pathlib
import time

from utils import get_collection, show_collection, nameof, mklist


# Modules used for EDA
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib.patches import Rectangle


# Modules used for image processing
import cv2

from collections import Counter
from utils import crop_img, chi_osc, extract_img_data, get_img_rgb
from utils import resize_img, reduce_col_palette, whitespace


# Modules used for ML
from sklearn.cluster import KMeans
from utils import color_quant

In [128]:
### TODO ### Import a class from a module

# For a better pd.DataFrame visualization
class display(object):
    '''This class was found in 'Python Data Science Handbook' by jakevdp (Jake Vanderplas),
    which you can access though his GitHub repository
    (https://github.com/jakevdp/PythonDataScienceHandbook)'''
    
    template = '''<div style="float: left; padding: 10px;">
                  <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
                  </div>'''
    
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_()) for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a)) for a in self.args)

In [104]:
# Modules settings
%matplotlib inline

___
### UPDATE UTILS

In [105]:
# This cell only needs to be executed to update utils
# if modified after been imported

# %run utils

___
### BASIC EDA

In [106]:
raw_data = pd.read_csv('./data/raw_museum/raw_data.csv', names=['img_ID', 'artist', 'height', 'width'])

raw_data.head(), raw_data.shape

(                img_ID      artist  height  width
 0  9223372032559824886  caravaggio     559    474
 1               186636  caravaggio     900    863
 2               186724  caravaggio     800    541
 3               186639  caravaggio    3239   4501
 4               186671  caravaggio     912   1200,
 (5233, 4))

In [107]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5233 entries, 0 to 5232
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   img_ID  5233 non-null   int64 
 1   artist  5233 non-null   object
 2   height  5233 non-null   int64 
 3   width   5233 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 163.7+ KB


In [108]:
raw_data.nunique()

img_ID    5233
artist      14
height    1609
width     1510
dtype: int64

In [109]:
# Get all paths to .jpg
extensions = ['.jpg']

raw_collection = get_collection(path='./images/raw_museum', extensions=extensions)
large_collection = get_collection(path='./images/large_museum', extensions=extensions)
mid_collection = get_collection(path='./images/mid_museum', extensions=extensions)
mid_sqr_collection = get_collection(path='./images/mid_sqr_museum', extensions=extensions)
low_sqr_collection = get_collection(path='./images/low_sqr_museum', extensions=extensions)

collections = [large_collection, mid_collection, mid_sqr_collection, low_sqr_collection]

print(f'{len(raw_collection)} images in raw_collection')
print(f'{len(large_collection)} images in large_collection')
print(f'{len(mid_collection)} images in mid_collection')
print(f'{len(mid_sqr_collection)} images in mid_sqr_collection')
print(f'{len(low_sqr_collection)} images in low_sqr_collection:')

5468 images in raw_collection
4820 images in large_collection
4594 images in mid_collection
4596 images in mid_sqr_collection
4485 images in low_sqr_collection:


In [110]:
# Get all paths to .csv files
extensions = ['.csv']

large_data = get_collection(path='./data/large_museum', extensions=extensions)
mid_data = get_collection(path='./data/mid_museum', extensions=extensions)
mid_sqr_data = get_collection(path='./data/mid_sqr_museum', extensions=extensions)
low_sqr_data = get_collection(path='./data/low_sqr_museum', extensions=extensions)

# Turn paths into str
large_data = [str(i) for i in large_data]
mid_data = [str(i) for i in mid_data]
mid_sqr_data = [str(i) for i in mid_sqr_data]
low_sqr_data = [str(i) for i in low_sqr_data]

# Build museums DataFrame
columns_names=['img_ID', 'artist', 'height', 'width', 'whitespace', 'chiaroscuro',
               'color_01', 'color_02', 'color_03', 'color_04', 'color_05', 
               'color_06', 'color_07', 'color_08', 'color_09', 'color_10']

large_museum = pd.concat((pd.read_csv(file, names=columns_names) for file in large_data), ignore_index=True)
mid_museum = pd.concat((pd.read_csv(file, names=columns_names) for file in mid_data), ignore_index=True)
mid_s_museum = pd.concat((pd.read_csv(file, names=columns_names) for file in mid_sqr_data), ignore_index=True)
low_s_museum = pd.concat((pd.read_csv(file, names=columns_names) for file in low_sqr_data), ignore_index=True)

museums = [large_museum, mid_museum, mid_s_museum, low_s_museum]

In [111]:
[museum.shape for museum in museums]

[(4815, 16), (4649, 16), (4476, 16), (4600, 16)]

In [112]:
[museum['img_ID'].nunique() for museum in museums]

[4815, 4649, 4476, 4600]

I'll keep only the common images to all museums

In [113]:
A = large_museum['img_ID'].isin(mid_museum['img_ID'])
B = large_museum['img_ID'].isin(mid_s_museum['img_ID'])
C = large_museum['img_ID'].isin(low_s_museum['img_ID'])

large_museum = large_museum[A & B & C]

In [114]:
A = mid_museum['img_ID'].isin(large_museum['img_ID'])
B = mid_museum['img_ID'].isin(mid_s_museum['img_ID'])
C = mid_museum['img_ID'].isin(low_s_museum['img_ID'])

mid_museum = mid_museum[A & B & C]

In [115]:
A = mid_s_museum['img_ID'].isin(large_museum['img_ID'])
B = mid_s_museum['img_ID'].isin(mid_museum['img_ID'])
C = mid_s_museum['img_ID'].isin(low_s_museum['img_ID'])

mid_s_museum = mid_s_museum[A & B & C]

In [116]:
A = low_s_museum['img_ID'].isin(large_museum['img_ID'])
B = low_s_museum['img_ID'].isin(mid_museum['img_ID'])
C = low_s_museum['img_ID'].isin(mid_s_museum['img_ID'])

low_s_museum = low_s_museum[A & B & C]

In [117]:
# Update museums
museums = [large_museum, mid_museum, mid_s_museum, low_s_museum]

[museum.shape for museum in museums]

[(4271, 16), (4271, 16), (4271, 16), (4271, 16)]

I'll not only drop the rows but also the images

In [118]:
# IMG_ID are not int64 type but object type, so I'll cast it
for museum in museums:
    museum['img_ID'] = museum['img_ID'].astype(str, errors='ignore')

In [119]:
large_museum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4271 entries, 0 to 4637
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   img_ID       4271 non-null   object 
 1   artist       4271 non-null   object 
 2   height       4271 non-null   int64  
 3   width        4271 non-null   int64  
 4   whitespace   4271 non-null   float64
 5   chiaroscuro  4271 non-null   float64
 6   color_01     4271 non-null   object 
 7   color_02     4271 non-null   object 
 8   color_03     4271 non-null   object 
 9   color_04     4271 non-null   object 
 10  color_05     4271 non-null   object 
 11  color_06     4271 non-null   object 
 12  color_07     4271 non-null   object 
 13  color_08     4271 non-null   object 
 14  color_09     4271 non-null   object 
 15  color_10     4271 non-null   object 
dtypes: float64(2), int64(2), object(12)
memory usage: 567.2+ KB


In [121]:
for collection in collections:
    collection_name = nameof(collection, globals())
    del_images = 0
    
    for image in collection:
        img_ID = str(image).split('/')[-1].split('.')[0]
        
        # I can use any museum as all have the same images
        if large_museum['img_ID'][large_museum['img_ID'].str.contains(img_ID)].any():
            continue
        elif os.path.exists(image):
            os.remove(image)
            del_images += 1
        
    print(f'{del_images} images deleted from {collection_name}')

544 images deleted from large_collection
318 images deleted from mid_collection
320 images deleted from mid_sqr_collection
209 images deleted from low_sqr_collection


In [124]:
# Update collections
extensions = ['.jpg']

raw_collection = get_collection(path='./images/raw_museum', extensions=extensions)
large_collection = get_collection(path='./images/large_museum', extensions=extensions)
mid_collection = get_collection(path='./images/mid_museum', extensions=extensions)
mid_sqr_collection = get_collection(path='./images/mid_sqr_museum', extensions=extensions)
low_sqr_collection = get_collection(path='./images/low_sqr_museum', extensions=extensions)

collections = [large_collection, mid_collection, mid_sqr_collection, low_sqr_collection]

print(f'{len(raw_collection)} images in raw_collection')
print(f'{len(large_collection)} images in large_collection')
print(f'{len(mid_collection)} images in mid_collection')
print(f'{len(mid_sqr_collection)} images in mid_sqr_collection')
print(f'{len(low_sqr_collection)} images in low_sqr_collection:')

5468 images in raw_collection
4276 images in large_collection
4276 images in mid_collection
4276 images in mid_sqr_collection
4276 images in low_sqr_collection:


___

At this point I will work with the large_museum as it contains more information and has no missings

In [140]:
def data_report(df):
    # Get names
    cols = pd.DataFrame(df.columns.values, columns=['COL_N'])
    # Get types
    types = pd.DataFrame(df.dtypes.values, columns=["DATA_TYPE"])
    
    # Get missings
    percent_missing = round(df.isnull().sum()*100/len(df),2)
    percent_missing_df = pd.DataFrame(percent_missing.values, columns = ["MISSINGS (%)"])

    # Get unique values
    unicos = pd.DataFrame(df.nunique().values, columns = ["UNIQUE_VALUES"])

    # Get cardinality
    percent_cardin = round(unicos["UNIQUE_VALUES"]*100/len(df),2)
    percent_cardin_df = pd.DataFrame(percent_cardin.values, columns = ["CARDIN (%)"])
      
    # Concat
    concatenado = pd.concat([cols, types, percent_missing_df, unicos, percent_cardin_df], axis=1)
    concatenado.set_index('COL_N', drop=True, inplace=True)

    return concatenado.T


l_repo = data_report(large_museum)
m_repo = data_report(mid_museum)
ms_repo = data_report(mid_s_museum)
ls_repo = data_report(low_s_museum)

display('l_repo', 'm_repo', 'ms_repo', 'ls_repo')

l_repo
COL_N          img_ID  artist height  width whitespace chiaroscuro color_01  \
DATA_TYPE      object  object  int64  int64    float64     float64   object   
MISSINGS (%)      0.0     0.0    0.0    0.0        0.0         0.0      0.0   
UNIQUE_VALUES    4271      14      1    442        248        3871       87   
CARDIN (%)      100.0    0.33   0.02  10.35       5.81       90.63     2.04   

COL_N         color_02 color_03 color_04 color_05 color_06 color_07 color_08  \
DATA_TYPE       object   object   object   object   object   object   object   
MISSINGS (%)       0.0      0.0      0.0      0.0      0.0      0.0      0.0   
UNIQUE_VALUES       83       88       87       88       90       90       89   
CARDIN (%)        1.94     2.06     2.04     2.06     2.11     2.11     2.08   

COL_N         color_09 color_10    ratio  
DATA_TYPE       object   object  float64  
MISSINGS (%)       0.0      0.0      0.0  
UNIQUE_VALUES       94       94      442  
CARDIN (%)         2.2      2.2    10.35  

m_repo
COL_N          img_ID  artist height  width whitespace chiaroscuro color_01  \
DATA_TYPE      object  object  int64  int64    float64     float64   object   
MISSINGS (%)      0.0     0.0    0.0    0.0        0.0         0.0      0.0   
UNIQUE_VALUES    4271      14      1      1          2        2380       75   
CARDIN (%)      100.0    0.33   0.02   0.02       0.05       55.72     1.76   

COL_N         color_02 color_03 color_04 color_05 color_06 color_07 color_08  \
DATA_TYPE       object   object   object   object   object   object   object   
MISSINGS (%)       0.0      0.0      0.0      0.0      0.0      0.0      0.0   
UNIQUE_VALUES       67       77       77       75       76       76       83   
CARDIN (%)        1.57      1.8      1.8     1.76     1.78     1.78     1.94   

COL_N         color_09 color_10  
DATA_TYPE       object   object  
MISSINGS (%)      0.02     0.02  
UNIQUE_VALUES       74       75  
CARDIN (%)        1.73     1.76  

ms_repo
COL_N          img_ID  artist height  width whitespace chiaroscuro color_01  \
DATA_TYPE      object  object  int64  int64    float64     float64   object   
MISSINGS (%)      0.0     0.0    0.0    0.0        0.0         0.0      0.0   
UNIQUE_VALUES    4271      14      1      1          2        2380       74   
CARDIN (%)      100.0    0.33   0.02   0.02       0.05       55.72     1.73   

COL_N         color_02 color_03 color_04 color_05 color_06 color_07 color_08  \
DATA_TYPE       object   object   object   object   object   object   object   
MISSINGS (%)       0.0      0.0      0.0      0.0      0.0      0.0      0.0   
UNIQUE_VALUES       73       80       79       73       78       77       79   
CARDIN (%)        1.71     1.87     1.85     1.71     1.83      1.8     1.85   

COL_N         color_09 color_10  
DATA_TYPE       object   object  
MISSINGS (%)      0.02     0.02  
UNIQUE_VALUES       82       77  
CARDIN (%)        1.92      1.8  

ls_repo
COL_N          img_ID  artist height  width whitespace chiaroscuro color_01  \
DATA_TYPE      object  object  int64  int64    float64     float64   object   
MISSINGS (%)      0.0     0.0    0.0    0.0        0.0         0.0      0.0   
UNIQUE_VALUES    4271      14      1      1          2        1527       74   
CARDIN (%)      100.0    0.33   0.02   0.02       0.05       35.75     1.73   

COL_N         color_02 color_03 color_04 color_05 color_06 color_07 color_08  \
DATA_TYPE       object   object   object   object   object   object   object   
MISSINGS (%)       0.0      0.0      0.0     0.02     0.02     0.05     0.09   
UNIQUE_VALUES       69       69       68       67       72       70       75   
CARDIN (%)        1.62     1.62     1.59     1.57     1.69     1.64     1.76   

COL_N         color_09 color_10  
DATA_TYPE       object   object  
MISSINGS (%)      0.19     0.49  
UNIQUE_VALUES       80       77  
CARDIN (%)        1.87      1.8

Let's take a closer look

In [141]:
display('large_museum', 'l_repo')

,img_ID,artist,height,width,whitespace,chiaroscuro,color_01,color_02,color_03,color_04,color_05,color_06,color_07,color_08,color_09,color_10,ratio
0,9223372032559824886,caravaggio,250,211,0.39810,0.00659,#3F0000,#FFFF7F,#3F003F,#BF3F3F,#FFBF7F,#7F0000,#BF7F3F,#3F0000,#FFFFBF,#BFBFBF,1.184834
1,186636,caravaggio,250,239,0.39833,0.01178,#003F00,#BF7F7F,#3F0000,#FFBF7F,#7F0000,#3F7F3F,#3F003F,#FFFFFF,#BF7F00,#3F3F00,1.046025
2,186724,caravaggio,250,169,0.39763,0.01400,#3F0000,#BFBFBF,#7F3F7F,#003F3F,#003F00,#3F3F3F,#FFFFFF,#7F3F3F,#BF7F7F,#FFFF7F,1.479290
3,186639,caravaggio,250,347,0.39885,0.02416,#3F3F00,#BF7F00,#7F7F3F,#FFBF7F,#7F003F,#3F003F,#3F3F3F,#7F3F00,#3F3F3F,#BF7F3F,0.720461
4,186671,caravaggio,250,328,0.39878,0.01706,#7F003F,#BF7F7F,#00003F,#7F0000,#3F0000,#FFBFBF,#3F0000,#003F00,#7F7F3F,#FF7F3F,0.762195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4633,229217,velazquez,250,362,0.39890,0.03147,#000000,#7F7F7F,#00003F,#BF7F7F,#003F3F,#7F003F,#7F7F00,#3F3F00,#FFFFBF,#7F0000,0.690608
4634,229182,velazquez,250,192,0.39792,191.00000,#7F7F00,#000000,#BFBF00,#FF3F00,#BF0000,#7F0000,#FFBF00,#FF003F,#7F3F00,#FF7F00,1.302083
4635,229183,velazquez,250,204,0.39804,0.00714,#BFBF7F,#3F0000,#003F3F,#7F7F00,#000000,#FFBFBF,#3F3F3F,#3F7F7F,#BF3F3F,#7F0000,1.225490
4636,9223372032559882013,velazquez,250,138,0.39710,0.01824,#BF7F7F,#3F0000,#7F3F00,#003F00,#BF3F00,#7F7F7F,#3F3F3F,#FFFFBF,#3F7F00,#003F00,1.811594


In [144]:
large_museum['ratio'] = large_museum['height'] / large_museum['width']

In [146]:
large_museum.drop(labels='height', axis=1, inplace=True)

In [147]:
large_museum.head()

,img_ID,artist,width,whitespace,chiaroscuro,color_01,color_02,color_03,color_04,color_05,color_06,color_07,color_08,color_09,color_10,ratio
0,9223372032559824886,caravaggio,211,0.39810,0.00659,#3F0000,#FFFF7F,#3F003F,#BF3F3F,#FFBF7F,#7F0000,#BF7F3F,#3F0000,#FFFFBF,#BFBFBF,1.184834
1,186636,caravaggio,239,0.39833,0.01178,#003F00,#BF7F7F,#3F0000,#FFBF7F,#7F0000,#3F7F3F,#3F003F,#FFFFFF,#BF7F00,#3F3F00,1.046025
2,186724,caravaggio,169,0.39763,0.01400,#3F0000,#BFBFBF,#7F3F7F,#003F3F,#003F00,#3F3F3F,#FFFFFF,#7F3F3F,#BF7F7F,#FFFF7F,1.479290
3,186639,caravaggio,347,0.39885,0.02416,#3F3F00,#BF7F00,#7F7F3F,#FFBF7F,#7F003F,#3F003F,#3F3F3F,#7F3F00,#3F3F3F,#BF7F3F,0.720461
4,186671,caravaggio,328,0.39878,0.01706,#7F003F,#BF7F7F,#00003F,#7F0000,#3F0000,#FFBFBF,#3F0000,#003F00,#7F7F3F,#FF7F3F,0.762195


In [148]:
corr = np.abs(large_museum.corr()['artist']).sort_values(ascending=False)
print(corr)

# Features con menos de 0.1 de correlación vs el target
bad_corr_feat = corr[corr < 0.1].index.values

# # Filtramos las columnas buenas
# large_museum.drop(columns=bad_corr_feat, inplace=True)

KeyError: 'artist'